In [ ]:
import os 
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import sys
sys.path.append("..")

from make_dir import mkdir
from load_yaml import get_yaml

import models.burgers_net as solutions
import equations.burgers_eqn as equation

from burgers_dataset import Sampler
import burgers_solver as solver 

from math import pi

import matplotlib.pyplot as plt
import time


In [ ]:
# load config
current_path = os.path.abspath(".")
yaml_path = os.path.join(current_path, "burgers.yaml")
Config = get_yaml(yaml_path)

# load reference data
absolute_path = os.path.abspath("..")
ref_path = os.path.join(absolute_path, "data/burgers.npz")
ref = np.load(ref_path)
freq = 1
ref_u = ref["density"][::freq].astype("float32").reshape(-1, 1) # shape: (256, 1)


In [ ]:
time_dimension = Config["physical_config"]["time_dimension"]
space_dimension = Config["physical_config"]["space_dimension"]
d_in = time_dimension + space_dimension
layers_u = Config["model_config"]["units_u"]

# build neural networks for u
Model_u = "solutions.Model_" + \
    "{}".format(Config["model_config"]["neural_network_type"])
Model_u = eval(Model_u)

model_u = Model_u(input_size = d_in, layers = layers_u, output_size = 1)

device_ids = Config["model_config"]["device_ids"]
device = torch.device("cuda:{:d}".format(device_ids[0]) if torch.cuda.is_available() else "cpu")

if torch.cuda.device_count() > 1:    
    model_u = nn.DataParallel(model_u, device_ids = device_ids)
    
model_u.to(device)



In [ ]:
# number of paramerters
param_num = sum(neural.numel() for neural in model_u.parameters())
print("Number of paramerters for networks u is: {:6d}. ".format(param_num))


In [ ]:
solutions.Xavier_initi(model_u)


In [ ]:
# make plot
def plot(iter):
    xmin = Config["physical_config"]["x_range"][0]
    xmax = Config["physical_config"]["x_range"][1]
    tmax = Config["physical_config"]["t_range"][1]

    nx = 256
    ref_x = torch.linspace(xmin, xmax, nx).reshape(-1, 1) 
    ref_t = tmax * torch.ones((nx, 1))  

    model_u = Sol
    u_approx = model_u(torch.Tensor(torch.cat([ref_t, ref_x], axis=-1)).to(device)).cpu().detach().numpy()

    plt.plot(ref_x, ref_u, "r", label = "density")
    plt.plot(ref_x, u_approx, "r*", markevery= 4, label = "approx density")
    plt.grid()
    plt.legend()
    plt.xlabel("x")
    plt.ylabel("u")
    plt.title("Approximate density and reference solution")
    plt.savefig("./figure/solution_iter_{:d}.pdf".format(iter))
    # plt.show()
    plt.close()

In [ ]:
# Set optimizer and learning rate decay
optimizer = optim.Adam([
    {'params': model_u.parameters()},
],  lr=Config["model_config"]["lr"])

scheduler = lr_scheduler.StepLR(
    optimizer, Config["model_config"]["stage_num"], Config["model_config"]["decay_rate"])


In [ ]:
Sol = model_u
bgk_eqn = equation.Burgers(config = Config, sol = Sol)

Iter = Config["model_config"]["iterations"] 
regularizers = Config["model_config"]["regularizers"]

loss_record, error_record = np.array([[]]).T, np.array([[]]*1).T

mkdir(file_dir = "./model_saved")
mkdir(file_dir = "./record")
mkdir(file_dir = "./figure")

time_start = time.time()
print('Begin training.')
print('')
for it in range(Iter):
    
    sampler = Sampler(Config)
    trainloader = [sampler.interior(), sampler.boundary(), sampler.initial()]
        
    risk, error = solver.train_step(sol = Sol,
                                    trainloader = trainloader, 
                                    equation = bgk_eqn,  
                                    regularizers = regularizers,
                                    optimizer = optimizer, 
                                    scheduler = scheduler,
                                    ref = ref)
    
    loss = risk["total_loss"]
    res_burgers_eqn = risk["burgers"]
    res_bc_u = risk["bc_u"]
    res_ic_u = risk["ic_u"]
    error = error["error"] 

    error = np.array(error, dtype=float).reshape(1, -1)
    loss_record = np.concatenate((loss_record, loss*np.ones((1, 1))), axis=0)
    error_record = np.concatenate((error_record, error), axis=0)

    lr = optimizer.state_dict()['param_groups'][0]['lr']
    
    if it % 100 == 0:
    
        print("[Iter: {:6d}/{:6d} - lr: {:.2e} and Loss: {:.2e}]".format(it + 1, Iter, lr, loss))
        print("[Error for density: {:.2e}]".format(float(error[:, 0])))
        print("[Burgers eqn: {:.2e}, Boundary - u: {:.2e}, Initial - u: {:.2e}]".format(res_burgers_eqn, res_bc_u, res_ic_u))
        
    if (it + 1) % 100 == 0:
        plot(iter = it + 1) 
        
    if np.max(error) < 1e-2:
        print("Iteration step: ", it)
        break

np.savez("./record/result.npz",
         loss=loss_record,
         error=error_record[:, 0])

solutions.save_param(model_u, path = './model_saved/model_u_params.pkl')

print("")
print("Finished training.")
time_end = time.time()
print("Total time is: {:.2e}".format(time_end - time_start), "seconds")


In [ ]:
# # load model
solutions.load_param(model_u, './model_saved/model_u_params.pkl')

Sol = model_u

In [ ]:
xmin = Config["physical_config"]["x_range"][0]
xmax = Config["physical_config"]["x_range"][1]
tmax = Config["physical_config"]["t_range"][1]
nx = 256
ref_x = torch.linspace(xmin, xmax, nx).reshape(-1, 1) 
ref_t = tmax * torch.ones((nx, 1))

In [ ]:
model_u = Sol
u_approx = model_u(torch.Tensor(torch.cat([ref_t, ref_x], axis=-1)).to(device)).cpu().detach().numpy()

plt.plot(ref_x, ref_u, "r", label="density")
plt.plot(ref_x, u_approx, "r*", markevery=4, label="approx density")
plt.grid()
plt.legend()
plt.xlabel("x")
plt.ylabel("macro")
plt.title("Approximate density and reference solution")
plt.savefig("./figure/solution.pdf")
plt.show()
plt.close()


In [ ]:
# # jupyter notebook to python
# try:   
#     !jupyter nbconvert --to python example.ipynb
# except:
#     pass